In [33]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [34]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
sns.set_style('darkgrid')
sns.set(font_scale=1.3)

In [35]:
import pip
pip.main(["install", "openpyxl"])

## **Read the Dataset**
1. Dataset consist of two columns, X and Y.
2. First 350 records are for training the model and remaining unlabelled data is for test purpose. 

In [36]:
df = pd.read_excel("/kaggle/input/itsc-ml-challenge-26jun/DATA_P.xlsx")

train = df.iloc[:350,1:] #segregate first 350 records as training data.
test = df.iloc[350:,1:] #remaining data to be treated as test.

In [37]:
display(train.info())

In [38]:
train.describe()

# **Exploratory Data Analysis**
1. The EDA is performed on both the independent variable and dependent variable, based on which we will analyse the data and check if any outliers are there or not. 
2. EDA will help us to select our model based on visualizations ie whether to pick tree based model or linear models.

In [39]:
#1. Histogram Plot
sns.histplot(x='X', data=train);

In [40]:
#2. Comparing both X, Y columns on Histogram plot. 

cols = 3
rows = 3
num_cols = train.select_dtypes(exclude='object').columns
fig = plt.figure( figsize=(cols*5, rows*5))
for i, col in enumerate(num_cols):
    
    ax=fig.add_subplot(rows,cols,i+1)
    
    sns.histplot(x = train[col], ax = ax)
    
fig.tight_layout()  
plt.show()

In [41]:
sns.heatmap(train.corr(), annot=True)

In [42]:
#3. This represents the box plot.

sns.boxplot(x=train['X'])

In [43]:
sns.boxplot(data=train.loc[:, ['X', 'Y']])

In [44]:
#4. Violin Plot for both X,Y variable 

sns.violinplot(x=train["X"], color='red')
sns.violinplot(x=train["Y"], color='yellow')

# **Creating a Train Test Split**
1. Train-Test split is 70:30 with random-state keeping as 33

In [45]:
from sklearn.model_selection import train_test_split

Xtrain,Xtest,ytrain,ytest = train_test_split(train[['X']], train[['Y']], test_size=0.3, random_state=33)

print(Xtrain.size, Xtest.size,  ytrain.size, ytest.size)

# **Model-1 Linear Regression**

In [46]:
#Linear Regression Model Training and Validation
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

m1 = LinearRegression(fit_intercept=True, normalize='deprecated', copy_X=True, 
                      n_jobs=5, positive=False)
m1.fit(Xtrain,ytrain)
ypred = m1.predict(Xtest)

s1 = (np.sqrt(mse(ytest, ypred)))
print(s1)

In [47]:
#Linear Regression Model prediction on Test dataset. 
pred = m1.predict(test[['X']])

sub1 = pd.DataFrame()
sub1['X'] = test['X']
sub1['Y'] = pred

sub1.to_csv('sub1.csv', index=False)

os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'sub1.csv')

# **Model-2 Random Forest Regressor**

In [48]:
from sklearn.ensemble import RandomForestRegressor

m2 = RandomForestRegressor(n_estimators=1550, criterion='squared_error', max_depth=33, 
                           min_samples_split=7, min_samples_leaf=7, min_weight_fraction_leaf=0.0, 
                           max_features=1.0, max_leaf_nodes=3, min_impurity_decrease=0.0, 
                           bootstrap=True, oob_score=False, n_jobs=3, random_state=33, 
                           verbose=0, warm_start=True, ccp_alpha=0.0, max_samples=1)
m2.fit(Xtrain,ytrain)
ypred = m2.predict(Xtest)

s2 = (np.sqrt(mse(ytest, ypred)))
print(s2)

In [49]:
#Random Forest Model prediction on Test dataset. 
pred = m2.predict(test[['X']])

sub2 = pd.DataFrame()
sub2['X'] = test['X']
sub2['Y'] = pred

sub2.to_csv('sub2.csv', index=False)

os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'sub2.csv')

# **Model-3 LightGBM Regressor**

In [50]:
from lightgbm import LGBMRegressor

m3 = LGBMRegressor(boosting_type='gbdt', num_leaves=6, max_depth=None, learning_rate=0.0017, 
                   n_estimators=1200, subsample_for_bin=200000, objective='regression', 
                   class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, 
                   subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, 
                   reg_lambda=0.0, random_state=33, n_jobs=3, importance_type='split')

m3.fit(Xtrain,ytrain)
ypred = m3.predict(Xtest)

s3 = (np.sqrt(mse(ytest, ypred)))
print(s3)

In [51]:
#LGBM Regressor Model prediction on Test dataset. 
pred = m2.predict(test[['X']])

sub3 = pd.DataFrame()
sub3['X'] = test['X']
sub3['Y'] = pred

sub3.to_csv('sub3.csv', index=False)

os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'sub3.csv')

# **Model-4 GradientBoostingRegressor**

In [52]:
from sklearn.ensemble import GradientBoostingRegressor

m4 = GradientBoostingRegressor(loss='squared_error', learning_rate=0.017, n_estimators=1400, 
                               subsample=1.0, criterion='friedman_mse', min_samples_split=2, 
                               min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=33, 
                               min_impurity_decrease=0.0, init=None, random_state=None, 
                               max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, 
                               warm_start=False, validation_fraction=0.1, n_iter_no_change=None, 
                               tol=0.0001, ccp_alpha=0.0)

m4.fit(Xtrain,ytrain)
ypred = m4.predict(Xtest)

s4 = (np.sqrt(mse(ytest, ypred)))
print(s4)

In [53]:
#Gradient Boosting predictions
pred = m4.predict(test[['X']])

sub4 = pd.DataFrame()
sub4['X'] = test['X']
sub4['Y'] = pred

sub4.to_csv('sub4.csv', index=False)

os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'sub4.csv')

# **Model-5 RANSAC Model**

In [54]:
from sklearn.linear_model import RANSACRegressor

m5 = RANSACRegressor(base_estimator=None, min_samples=None, residual_threshold=None, is_data_valid=None, is_model_valid=None, 
                     max_trials=1000, max_skips=np.inf, stop_n_inliers=np.inf, stop_score=np.inf, stop_probability=0.99, 
                     loss="absolute_error", random_state=33)

m5.fit(Xtrain,ytrain)
ypred = m5.predict(Xtest)

s5 = (np.sqrt(mse(ytest, ypred)))
print(s5)

In [55]:
#RANSAC predictions
pred = m5.predict(test[['X']])

sub5 = pd.DataFrame()
sub5['X'] = test['X']
sub5['Y'] = pred

sub4.to_csv('sub5.csv', index=False)

os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'sub5.csv')

# **Results Summary of all the Models above**

In [56]:
#Result Evaluation
scores = pd.DataFrame()
scores['Algorithm'] = ['Linear Regression', 'Random Forest', 'LightGBM', 'GradientBoosting', 'RANSAC']
scores['RMSE_Score'] = [s1,s2,s3,s4,s5]
scores['submission_file'] = [FileLink(r'sub1.csv'),FileLink(r'sub2.csv'),FileLink(r'sub3.csv'),
                             FileLink(r'sub4.csv'), FileLink(r'sub5.csv')] #prediction on test data

display(scores)

print(" ")
print(" ")
print("******* Winner Model *******")
print(" ")
print(" ")

display(scores[scores['RMSE_Score'] == min(scores['RMSE_Score'])])

In [57]:
winner = scores[scores['RMSE_Score'] == min(scores['RMSE_Score'])]['submission_file']

In [58]:
plot_df = pd.read_csv("/kaggle/working/"+winner.values[0].path)

In [59]:
sns.lineplot(x = "X",
             y = "Y",
             data = plot_df, color='red');